In [16]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import streamlit as st
import matplotlib.pyplot as plt

# Load API key from .env file
load_dotenv()
BLS_API_KEY = os.getenv("BLS_API_KEY")


# BLS API endpoint
API_URL = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

def fetch_bls_data(series_id, start_year, end_year):
    """
    Fetch data from the BLS Public API for a given series ID and date range.

    Args:
        series_id (str): The BLS series ID.
        start_year (int): The start year for the data request.
        end_year (int): The end year for the data request.

    Returns:
        dict: Raw JSON response containing the BLS data.
    """
    headers = {"Content-Type": "application/json"}
    payload = {
        "seriesid": [series_id],
        "startyear": str(start_year),
        "endyear": str(end_year),
        "registrationkey": BLS_API_KEY
    }
    response = requests.post(API_URL, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

def process_bls_data(raw_data):
    """
    Process the raw BLS API data into a clean Pandas DataFrame.

    Args:
        raw_data (dict): The raw JSON response from the BLS API.

    Returns:
        pd.DataFrame: A DataFrame containing 'date' and 'value' columns.
    """
    try:
        # Extract the data series
        series = raw_data['Results']['series'][0]['data']

        # Convert the series to a DataFrame
        df = pd.DataFrame(series)

        # Format the 'date' column
        df['date'] = pd.to_datetime(df['year'] + '-' + df['period'].str[1:], errors='coerce')

        # Drop invalid dates and convert 'value' to numeric
        df = df.dropna(subset=['date']).sort_values(by='date')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')

        # Return only relevant columns
        return df[['date', 'value']]
    except Exception as e:
        raise Exception(f"Error processing data: {e}")

# App title
st.title("US Labor Statistics Dashboard")

# Sidebar inputs
st.sidebar.header("Data Configuration")
start_year = st.sidebar.number_input("Start Year", min_value=2000, max_value=2024, value=2023)
end_year = st.sidebar.number_input("End Year", min_value=2000, max_value=2024, value=2024)

# Series IDs and descriptions
series_info = {
    "CES0000000001": "Total Non-Farm Payrolls",
    "LNS14000000": "Unemployment Rate"
}

# Main dashboard display
for series_id, description in series_info.items():
    st.write(f"### {description}")
    try:
        # Fetch and process data
        raw_data = fetch_bls_data(series_id, start_year, end_year)
        df = process_bls_data(raw_data)

        # Display data table
        st.dataframe(df)

        # Display chart
        plt.figure(figsize=(10, 5))
        plt.plot(df['date'], df['value'], marker='o', linestyle='-')
        plt.title(description)
        plt.xlabel("Date")
        plt.ylabel("Value")
        plt.grid()
        st.pyplot(plt)
    except Exception as e:
        st.error(f"Error fetching data for {description}: {e}")



2024-12-18 01:55:20.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 01:55:20.434 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 01:55:20.440 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 01:55:20.444 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 01:55:20.447 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 01:55:20.450 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 01:55:20.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 01:55:20.456 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar